## Setting up 3 dataframes: LA County Hazard Data, FEMA data, and FEMA Shapefile data

In [68]:
import pandas as pd


In [67]:
from google.colab import drive

drive.mount('/content/drive') # allow access to Drive

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [69]:
import os
import geopandas as gpd


# list files and directories in your Google Drive root
# os.listdir('/content/drive/My Drive/BTTAI Group 46 - PREPARED/')
os.listdir('/content/drive/My Drive/Break Through Tech AI @ MIT/AI Studio Project/BTTAI Group 46 - PREPARED/') # my root

shapefile_path = ('/content/drive/My Drive/BTTAI Group 46 - PREPARED/NRI_Shapefile_CensusTracts.zip') # general path
shapefile_path = ('/content/drive/My Drive/Break Through Tech AI @ MIT/AI Studio Project/BTTAI Group 46 - PREPARED/NRI_Shapefile_CensusTracts.zip') # my path
gdf = gpd.read_file(shapefile_path)


In [ ]:
import gspread
from gspread_dataframe import get_as_dataframe, set_with_dataframe

from google.colab import auth
from google.auth import default

#authenticate and create a client to interact with Google Sheets
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

In [70]:
la_data_sheet = gc.open('Los Angeles Hazard Data')
fema_data_sheet = gc.open('cleaned_NRI_Table_CensusTracts_California')
worksheet = fema_data_sheet.sheet1  # Access the first sheet

# Convert the worksheet to a pandas DataFrame
fema_data_df = get_as_dataframe(worksheet)
print(fema_data_df.head())

     OID_        NRI_ID       STATE STATEABBRV  STATEFIPS   COUNTY COUNTYTYPE  \
0  4201.0  T06001400100  California         CA        6.0  Alameda     County   
1  4202.0  T06001400200  California         CA        6.0  Alameda     County   
2  4203.0  T06001400300  California         CA        6.0  Alameda     County   
3  4204.0  T06001400400  California         CA        6.0  Alameda     County   
4  4205.0  T06001400500  California         CA        6.0  Alameda     County   

   COUNTYFIPS  STCOFIPS     TRACT  ...  WNTW_ALRB  WNTW_ALRP  WNTW_ALRA  \
0         1.0    6001.0  400100.0  ...        0.0        0.0        0.0   
1         1.0    6001.0  400200.0  ...        0.0        0.0        0.0   
2         1.0    6001.0  400300.0  ...        0.0        0.0        0.0   
3         1.0    6001.0  400400.0  ...        0.0        0.0        0.0   
4         1.0    6001.0  400500.0  ...        0.0        0.0        0.0   

   WNTW_ALR_NPCTL  WNTW_RISKV  WNTW_RISKS  WNTW_RISKR     NRI_

In [ ]:
tabs = la_data_sheet.worksheets() #list of all sheet names aka tabs in the la hazard data
la_data_df = {}
for tab in tabs:
  la_data_df[tab.title] = get_as_dataframe(tab)
  print(tab.title)

Sheet1
Wildfire_RCP45
Historic_Earthquakes
Tsunami_Tide_Gauges
Extreme_Heat
Los_Angeles_County_CVA_Social_Sensitivity_Index
Extreme_Precipitation_Low_Emissions_RCP45
LACoFD_Historic_Fire_Perimeters_(Feature_Layer)
LACoFD_Fire_Hazard_Severity_Zones_%E2%80%93_SRA_(Feature_Layer)
Social_Vulnerability_Index_2020_3398216233094230935
EJSM_Scores
Wildfire_RCP_8.5
LACoFD_Fire_Hazard_Severity_Zones_%E2%80%93_LRA_(Feature_Layer)
Extreme_Precipitation_Low_Emissions_RCP_8.5
Extreme_Heat_Low_Emissions_RCP_45


In [ ]:
fema_data_df.columns

Index(['OID_', 'NRI_ID', 'STATE', 'STATEABBRV', 'STATEFIPS', 'COUNTY',
       'COUNTYTYPE', 'COUNTYFIPS', 'STCOFIPS', 'TRACT',
       ...
       'WNTW_ALRB', 'WNTW_ALRP', 'WNTW_ALRA', 'WNTW_ALR_NPCTL', 'WNTW_RISKV',
       'WNTW_RISKS', 'WNTW_RISKR', 'NRI_VER', 'Unnamed: 445', 'Unnamed: 446'],
      dtype='object', length=447)

## Renaming columns across FEMA data and LA Hazard data


In [71]:
# Renaming the census tract column in FEMA data and columns in LA Hazard data for consistency
fema_data_df.rename(columns={'TRACT': 'Census Tract'}, inplace = True)
la_data_df['Los_Angeles_County_CVA_Social_Sensitivity_Index'].rename(columns={'Census_Tract': 'Census Tract'}, inplace = True)
la_data_df['Social_Vulnerability_Index_2020_3398216233094230935'].rename(columns={'Tract 2020': 'Census Tract'}, inplace = True)
la_data_df['EJSM_Scores'].rename(columns={'Tract_1': 'Census Tract'}, inplace = True)

In [72]:
fema_data_df= fema_data_df[fema_data_df['COUNTY'] == 'Los Angeles'] #only keeping LA data

In [73]:
# Renaming other columns in FEMA data for clarity
fema_data_df.rename(columns={'POPULATION': 'Population'}, inplace = True)
fema_data_df.rename(columns={'SOVI_SCORE': 'Social Vulnerability Index'}, inplace = True)


## Creating sentences from FEMA data

In [76]:
# Process FEMA dataset to create sentences
def create_sentence(row):
  # including certain rows from the FEMA dataset
    census_tract = row['Census Tract']
    population = row['Population']
    svi = row['Social Vulnerability Index']
    fips = row['TRACTFIPS']
    county = row['COUNTY']
    risk_value = row['RISK_VALUE']


    return (
        f"In Census Tract {census_tract} (FIPS {fips}), located in {county}, the total population is {population}, "
        f"with a risk value of {risk_value} and a Social Vulnerability Index score of {svi}."
    )
# Create the sentences
fema_data_df['sentence'] = fema_data_df.apply(create_sentence, axis=1)
fema_data_df['sentence']

,sentence
1377,"In Census Tract 101110.0 (FIPS 6037101110.0), ..."
1378,"In Census Tract 101122.0 (FIPS 6037101122.0), ..."
1379,"In Census Tract 101220.0 (FIPS 6037101220.0), ..."
1380,"In Census Tract 101221.0 (FIPS 6037101221.0), ..."
1381,"In Census Tract 101222.0 (FIPS 6037101222.0), ..."
...,...
3867,"In Census Tract 980035.0 (FIPS 6037980035.0), ..."
3868,"In Census Tract 980036.0 (FIPS 6037980036.0), ..."
3869,"In Census Tract 980037.0 (FIPS 6037980037.0), ..."
3870,"In Census Tract 980038.0 (FIPS 6037980038.0), ..."


## Standardizing census tract format across FEMA and La Hazard data


In [77]:
fema_data_df['Census Tract'] = fema_data_df['Census Tract'].astype(str)

# Convert 'Census Tract' to string in LA hazard data
la_data_df['Los_Angeles_County_CVA_Social_Sensitivity_Index']['Census Tract'] = la_data_df['Los_Angeles_County_CVA_Social_Sensitivity_Index']['Census Tract'].astype(str)
social_vulnerability = la_data_df['Social_Vulnerability_Index_2020_3398216233094230935']
social_vulnerability['Census Tract'] = social_vulnerability['Census Tract'].astype(str)
ejsm_scores = la_data_df['EJSM_Scores']
ejsm_scores['Census Tract'] = ejsm_scores['Census Tract'].astype(str)

In [78]:
la_data_df['Los_Angeles_County_CVA_Social_Sensitivity_Index']['Census Tract'] = la_data_df['Los_Angeles_County_CVA_Social_Sensitivity_Index']['Census Tract'].str.replace('Census Tract ', '')

#extracting last 6 digits of EJSM scores
la_data_df['EJSM_Scores']['Census Tract'] = la_data_df['EJSM_Scores']['Census Tract'].str[-6:]

In [79]:
# removing decimals from any columns that have it
for tab_name, df in la_data_df.items():
  if 'Census Tract' in df.columns:
    df['Census Tract'] = df['Census Tract'].astype(str).str.replace('.', '').str[:6].str.zfill(6)

In [82]:
#confirming tabs and printing column info for LA County Hazard data
for tab_name, df in la_data_df.items():
  print(f"Tab Name: {tab_name}")
  print(df.columns)
  # print(df.head())

Tab Name: Sheet1
Index(['Unnamed: 0', 'Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4',
       'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9',
       'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12', 'Unnamed: 13',
       'Unnamed: 14', 'Unnamed: 15', 'Unnamed: 16', 'Unnamed: 17',
       'Unnamed: 18', 'Unnamed: 19', 'Unnamed: 20', 'Unnamed: 21',
       'Unnamed: 22', 'Unnamed: 23', 'Unnamed: 24', 'Unnamed: 25'],
      dtype='object')
Tab Name: Wildfire_RCP45
Index(['OBJECTID', 'Geo_UID', 'Grid_Num', 'longitude', 'latitude', 'Baseline',
       'F_2050', 'F_2080', 'Shape_Length', 'Shape_Area', 'Mid_Cent',
       'Late_Cent', 'Shape__Area', 'Shape__Length', 'sentence'],
      dtype='object')
Tab Name: Historic_Earthquakes
Index(['X', 'Y', 'OBJECTID', 'source', 'ext_id', 'cat1', 'cat2', 'org_name',
       'Name', 'info2', 'post_id', 'link', 'use_type', 'latitude', 'longitude',
       'date_updated', 'POINT_X', 'POINT_Y', 'sentence'],
      dtype='object')
Tab Name: 

## Merging FEMA data and LA Hazard data on census tract

In [117]:
# # Merging the FEMA Data with LA Hazard Data (matching 'Census Tract')

# merged_data = pd.merge(la_data_df['Sheet1'], fema_data_df[['Census Tract', 'sentence']],
#                        on='Census Tract', how='left')

# Merging la and fema data on census tract

for tab in tabs:

    # Check if the 'Census Tract' column exists in the current tab
    if 'Census Tract' in la_data_df[tab.title].columns:

        # Merge based on Census Tract column
        merged_data = pd.merge(fema_data_df, la_data_df[tab.title], on='Census Tract', how='left')
        print(f"Merged data for tab: {tab.title}")
        print(merged_data.head())
    else:
        print(f"No Census Tract column found in tab: {tab.title}")


No Census Tract column found in tab: Sheet1
No Census Tract column found in tab: Wildfire_RCP45
No Census Tract column found in tab: Historic_Earthquakes
No Census Tract column found in tab: Tsunami_Tide_Gauges
No Census Tract column found in tab: Extreme_Heat
Merged data for tab: Los_Angeles_County_CVA_Social_Sensitivity_Index
     OID_        NRI_ID       STATE STATEABBRV  STATEFIPS       COUNTY  \
0  5578.0  T06037101110  California         CA        6.0  Los Angeles   
1  5579.0  T06037101122  California         CA        6.0  Los Angeles   
2  5580.0  T06037101220  California         CA        6.0  Los Angeles   
3  5581.0  T06037101221  California         CA        6.0  Los Angeles   
4  5582.0  T06037101222  California         CA        6.0  Los Angeles   

  COUNTYTYPE  COUNTYFIPS  STCOFIPS Census Tract     TRACTFIPS  Population_x  \
0     County        37.0    6037.0     101110.0  6.037101e+09        4926.0   
1     County        37.0    6037.0     101122.0  6.037101e+09      

## Generating more sentences from FEMA Shapefile data

---



In [118]:
gdf.rename(columns={'TRACT': 'Census Tract'}, inplace = True) #renaming gdf column to "Census Tract"
la_gdf = gdf[gdf['COUNTY'] == 'Los Angeles'] #only keeping LA data
# print(la_gdf.head())
print(la_gdf.columns)

KeyError: 'COUNTY'

In [119]:
# # Converting whole geodataframe to string
# la_gdf_string = la_gdf.to_string()
# print(la_gdf)

In [120]:
# Adjust display width to show all columns
pd.set_option('display.max_columns', None)

# Print all column names
print(la_gdf.columns.tolist())

['NRI_ID', 'STATE', 'STATEABBRV', 'STATEFIPS', 'COUNTY', 'COUNTYTYPE', 'COUNTYFIPS', 'STCOFIPS', 'Census Tract', 'TRACTFIPS', 'Population', 'BUILDVALUE', 'AGRIVALUE', 'AREA', 'RISK_VALUE', 'RISK_SCORE', 'RISK_RATNG', 'RISK_SPCTL', 'EAL_SCORE', 'EAL_RATNG', 'EAL_SPCTL', 'EAL_VALT', 'EAL_VALB', 'EAL_VALP', 'EAL_VALPE', 'EAL_VALA', 'ALR_VALB', 'ALR_VALP', 'ALR_VALA', 'ALR_NPCTL', 'ALR_VRA_NP', 'Social Vulnerability Index', 'SOVI_RATNG', 'SOVI_SPCTL', 'RESL_SCORE', 'RESL_RATNG', 'RESL_SPCTL', 'RESL_VALUE', 'CRF_VALUE', 'AVLN_EVNTS', 'AVLN_AFREQ', 'AVLN_EXP_A', 'AVLN_EXPB', 'AVLN_EXPP', 'AVLN_EXPPE', 'AVLN_EXPT', 'AVLN_HLRB', 'AVLN_HLRP', 'AVLN_HLRR', 'AVLN_EALB', 'AVLN_EALP', 'AVLN_EALPE', 'AVLN_EALT', 'AVLN_EALS', 'AVLN_EALR', 'AVLN_ALRB', 'AVLN_ALRP', 'AVLN_ALR_N', 'AVLN_RISKV', 'AVLN_RISKS', 'AVLN_RISKR', 'CFLD_EVNTS', 'CFLD_AFREQ', 'CFLD_EXP_A', 'CFLD_EXPB', 'CFLD_EXPP', 'CFLD_EXPPE', 'CFLD_EXPT', 'CFLD_HLRB', 'CFLD_HLRP', 'CFLD_HLRR', 'CFLD_EALB', 'CFLD_EALP', 'CFLD_EALPE', 'CFLD_EALT

In [121]:
#filtering out tabs with lat and long in LA Hazard data
tabs_with_latlong = []
for tab in tabs:
  if 'latitude' in la_data_df[tab.title].columns and 'longitude' in la_data_df[tab.title].columns:
    tabs_with_latlong.append(tab.title)
print(tabs_with_latlong)

['Wildfire_RCP45', 'Historic_Earthquakes', 'Tsunami_Tide_Gauges', 'Extreme_Heat', 'Extreme_Precipitation_Low_Emissions_RCP45', 'Wildfire_RCP_8.5', 'Extreme_Precipitation_Low_Emissions_RCP_8.5', 'Extreme_Heat_Low_Emissions_RCP_45']


In [122]:
#List to store merged data
all_merged_data = []

In [123]:
for tab_title in tabs_with_latlong:
  df = la_data_df[tab_title]
  #convert lat/long to geometry
  gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df['longitude'], df['latitude']))
  #spatial join with LA county shapefile
  gdf = gpd.sjoin(gdf, la_gdf[['Census Tract', 'geometry']], how='left', predicate='within')
  all_merged_data.append(merged_data)

  print(f"Merged data for tab with lat/long: {tab_title}")
  print(gdf.head())

Merged data for tab with lat/long: Wildfire_RCP45
   OBJECTID           Geo_UID  Grid_Num  longitude  latitude  Baseline  \
0         1    32.75-118.4375     17562  -118.4375   32.7500    2.9133   
1         2   32.8125-118.375     17742  -118.3750   32.8125    4.2058   
2         3  32.8125-118.4375     17563  -118.4375   32.8125   19.1550   
3         4     32.8125-118.5     17384  -118.5000   32.8125   12.5058   
4         5  32.8125-118.5625     17205  -118.5625   32.8125    2.8242   

   F_2050  F_2080  Shape_Length    Shape_Area  Mid_Cent  Late_Cent  \
0   -1.34   -1.97      0.110417  3.124653e-04     1.573      0.943   
1   -1.87   -2.67      0.121772  5.804570e-04     2.336      1.536   
2   -6.92  -10.60      0.225823  3.189656e-03    12.235      8.555   
3   -4.22   -6.74      0.218413  2.354185e-03     8.286      5.766   
4   -1.05   -1.66      0.005118  9.255045e-07     1.774      1.164   

    Shape__Area  Shape__Length  \
0  4.606830e+06   12910.061310   
1  8.559597e+06 

<ipython-input-123-0eadf897f028>:6: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: None
Right CRS: EPSG:3857

  gdf = gpd.sjoin(gdf, la_gdf[['Census Tract', 'geometry']], how='left', predicate='within')
<ipython-input-123-0eadf897f028>:6: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: None
Right CRS: EPSG:3857

  gdf = gpd.sjoin(gdf, la_gdf[['Census Tract', 'geometry']], how='left', predicate='within')
<ipython-input-123-0eadf897f028>:6: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: None
Right CRS: EPSG:3857

  gdf = gpd.sjoin(gdf, la_gdf[['Census Tract', 'g

## Generating sentences from FEMA shapefile data, also includes latitude and longitude

In [91]:
# Save GeoDataFrame to CSV
la_gdf.to_csv('la_gdf.csv', index=False)

# Download the CSV
from google.colab import files
files.download('la_gdf.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [124]:
# Renaming columns in FEMA shapefile data for clarity
la_gdf.rename(columns={'POPULATION': 'Population'}, inplace = True)
la_gdf.rename(columns={'SOVI_SCORE': 'Social Vulnerability Index'}, inplace = True)

In [125]:
# Making sentences from FEMA shapefile data
# Process FEMA dataset to create sentences
# def create_sentence(row):
#   # including certain rows from the FEMA dataset
#     census_tract = row['Census Tract']
#     population = row['Population']
#     svi = row['Social Vulnerability Index']
#     fips = row['TRACTFIPS']
#     county = row['COUNTY']
#     risk_value = row['RISK_VALUE']


#     return (
#         f"In Census Tract {census_tract} (FIPS {fips}), located in {county}, the total population is {population}, "
#         f"with a risk value of {risk_value} and a Social Vulnerability Index score of {svi}."
#     )
# Create the sentences
la_gdf['sentence'] = la_gdf.apply(create_sentence, axis=1)
la_gdf['sentence']

,sentence
5577,"In Census Tract 101110 (FIPS 06037101110), loc..."
5578,"In Census Tract 101122 (FIPS 06037101122), loc..."
5579,"In Census Tract 101220 (FIPS 06037101220), loc..."
5580,"In Census Tract 101221 (FIPS 06037101221), loc..."
5581,"In Census Tract 101222 (FIPS 06037101222), loc..."
...,...
8067,"In Census Tract 980035 (FIPS 06037980035), loc..."
8068,"In Census Tract 980036 (FIPS 06037980036), loc..."
8069,"In Census Tract 980037 (FIPS 06037980037), loc..."
8070,"In Census Tract 980038 (FIPS 06037980038), loc..."


In [126]:
#merging la data with la county shapefile on census tract
for tab in tabs:

    # Check if the 'Census Tract' column exists in the current tab
    if 'Census Tract' in la_data_df[tab.title].columns:

        # Merge based on Census Tract column
        merged_data2 = pd.merge(la_data_df[tab.title], la_gdf[['Census Tract', 'geometry']], on='Census Tract', how='left')

        print(f"Merged data for tab: {tab.title}")
        # print(merged_data2.head())

        all_merged_data.append(merged_data2)
    else:
        print(f"No Census Tract column found in tab: {tab.title}")

No Census Tract column found in tab: Sheet1
No Census Tract column found in tab: Wildfire_RCP45
No Census Tract column found in tab: Historic_Earthquakes
No Census Tract column found in tab: Tsunami_Tide_Gauges
No Census Tract column found in tab: Extreme_Heat
Merged data for tab: Los_Angeles_County_CVA_Social_Sensitivity_Index
No Census Tract column found in tab: Extreme_Precipitation_Low_Emissions_RCP45
No Census Tract column found in tab: LACoFD_Historic_Fire_Perimeters_(Feature_Layer)
No Census Tract column found in tab: LACoFD_Fire_Hazard_Severity_Zones_%E2%80%93_SRA_(Feature_Layer)
Merged data for tab: Social_Vulnerability_Index_2020_3398216233094230935
Merged data for tab: EJSM_Scores
No Census Tract column found in tab: Wildfire_RCP_8.5
No Census Tract column found in tab: LACoFD_Fire_Hazard_Severity_Zones_%E2%80%93_LRA_(Feature_Layer)
No Census Tract column found in tab: Extreme_Precipitation_Low_Emissions_RCP_8.5
No Census Tract column found in tab: Extreme_Heat_Low_Emissions

In [127]:
all_merged_data

[        OID_        NRI_ID       STATE STATEABBRV  STATEFIPS       COUNTY  \
 0     5578.0  T06037101110  California         CA        6.0  Los Angeles   
 1     5579.0  T06037101122  California         CA        6.0  Los Angeles   
 2     5580.0  T06037101220  California         CA        6.0  Los Angeles   
 3     5581.0  T06037101221  California         CA        6.0  Los Angeles   
 4     5582.0  T06037101222  California         CA        6.0  Los Angeles   
 ...      ...           ...         ...        ...        ...          ...   
 2490  8068.0  T06037980035  California         CA        6.0  Los Angeles   
 2491  8069.0  T06037980036  California         CA        6.0  Los Angeles   
 2492  8070.0  T06037980037  California         CA        6.0  Los Angeles   
 2493  8071.0  T06037980038  California         CA        6.0  Los Angeles   
 2494  8072.0  T06037980039  California         CA        6.0  Los Angeles   
 
      COUNTYTYPE  COUNTYFIPS  STCOFIPS Census Tract     TRACTF

In [128]:
print(type(all_merged_data))  # Check if it's a list
# print(len(all_merged_data))  # See how many items are in the list
# print(all_merged_data[0])  # Check the structure of the first item

<class 'list'>


In [129]:
# df_cleaned = all_merged_data.drop(columns='geometry', errors='ignore')  # Drop geometry column if you don't need it

AttributeError: 'list' object has no attribute 'drop'

In [130]:
#combining merged data from both methods
final_merged_data = pd.concat(all_merged_data, ignore_index=True)
print(final_merged_data.columns)
final_merged_data.head()


Index(['OID_', 'NRI_ID', 'STATE', 'STATEABBRV', 'STATEFIPS', 'COUNTY',
       'COUNTYTYPE', 'COUNTYFIPS', 'STCOFIPS', 'Census Tract',
       ...
       'Sum of series for Household Characteristics theme',
       'Percentile ranking for Household Characteristics theme summary',
       'Sum of series for Racial and Ethnic Minority Status theme',
       'Percentile ranking for Racial and Ethnic Minority Status theme',
       'Sum of series for Housing Type/ Transportation theme',
       'Percentile ranking for Housing Type/ Transportation theme',
       'Sum of Series Themes', 'Overall Percentile Ranking', 'ObjectID',
       'Level of Vulnerability'],
      dtype='object', length=524)


,OID_,NRI_ID,STATE,STATEABBRV,STATEFIPS,COUNTY,COUNTYTYPE,COUNTYFIPS,STCOFIPS,Census Tract,TRACTFIPS,Population,BUILDVALUE,AGRIVALUE,AREA,RISK_VALUE,RISK_SCORE,RISK_RATNG,RISK_SPCTL,EAL_SCORE,EAL_RATNG,EAL_SPCTL,EAL_VALT,EAL_VALB,EAL_VALP,EAL_VALPE,EAL_VALA,ALR_VALB,ALR_VALP,ALR_VALA,ALR_NPCTL,ALR_VRA_NPCTL,Social Vulnerability Index,SOVI_RATNG,SOVI_SPCTL,RESL_SCORE,RESL_RATNG,RESL_SPCTL,RESL_VALUE,CRF_VALUE,AVLN_EVNTS,AVLN_AFREQ,AVLN_EXP_AREA,AVLN_EXPB,AVLN_EXPP,AVLN_EXPPE,AVLN_EXPT,AVLN_HLRB,AVLN_HLRP,AVLN_HLRR,AVLN_EALB,AVLN_EALP,AVLN_EALPE,AVLN_EALT,AVLN_EALS,AVLN_EALR,AVLN_ALRB,AVLN_ALRP,AVLN_ALR_NPCTL,AVLN_RISKV,AVLN_RISKS,AVLN_RISKR,CFLD_AFREQ,CFLD_EXP_AREA,CFLD_EXPB,CFLD_EXPP,CFLD_EXPPE,CFLD_EXPT,CFLD_HLRB,CFLD_HLRP,CFLD_HLRR,CFLD_EALB,CFLD_EALP,CFLD_EALPE,CFLD_EALT,CFLD_EALS,CFLD_EALR,CFLD_ALRB,CFLD_ALRP,CFLD_ALR_NPCTL,CFLD_RISKV,CFLD_RISKS,CFLD_RISKR,CWAV_EVNTS,CWAV_AFREQ,CWAV_EXP_AREA,CWAV_EXPB,CWAV_EXPP,CWAV_EXPPE,CWAV_EXPA,CWAV_EXPT,CWAV_HLRB,CWAV_HLRP,CWAV_HLRA,CWAV_HLRR,CWAV_EALB,CWAV_EALP,CWAV_EALPE,CWAV_EALA,CWAV_EALT,CWAV_EALS,CWAV_EALR,CWAV_ALRB,CWAV_ALRP,CWAV_ALRA,CWAV_ALR_NPCTL,CWAV_RISKV,CWAV_RISKS,CWAV_RISKR,DRGT_EVNTS,DRGT_AFREQ,DRGT_EXP_AREA,DRGT_EXPA,DRGT_EXPT,DRGT_HLRA,DRGT_HLRR,DRGT_EALA,DRGT_EALT,DRGT_EALS,DRGT_EALR,DRGT_ALRA,DRGT_ALR_NPCTL,DRGT_RISKV,DRGT_RISKS,DRGT_RISKR,ERQK_AFREQ,ERQK_EXP_AREA,ERQK_EXPB,ERQK_EXPP,ERQK_EXPPE,ERQK_EXPT,ERQK_HLRB,ERQK_HLRP,ERQK_HLRR,ERQK_EALB,ERQK_EALP,ERQK_EALPE,ERQK_EALT,ERQK_EALS,ERQK_EALR,ERQK_ALRB,ERQK_ALRP,ERQK_ALR_NPCTL,ERQK_RISKV,ERQK_RISKS,ERQK_RISKR,HAIL_EVNTS,HAIL_AFREQ,HAIL_EXP_AREA,HAIL_EXPB,HAIL_EXPP,HAIL_EXPPE,HAIL_EXPA,HAIL_EXPT,HAIL_HLRB,HAIL_HLRP,HAIL_HLRA,HAIL_HLRR,HAIL_EALB,HAIL_EALP,HAIL_EALPE,HAIL_EALA,HAIL_EALT,HAIL_EALS,HAIL_EALR,HAIL_ALRB,HAIL_ALRP,HAIL_ALRA,HAIL_ALR_NPCTL,HAIL_RISKV,HAIL_RISKS,HAIL_RISKR,HWAV_EVNTS,HWAV_AFREQ,HWAV_EXP_AREA,HWAV_EXPB,HWAV_EXPP,HWAV_EXPPE,HWAV_EXPA,HWAV_EXPT,HWAV_HLRB,HWAV_HLRP,HWAV_HLRA,HWAV_HLRR,HWAV_EALB,HWAV_EALP,HWAV_EALPE,HWAV_EALA,HWAV_EALT,HWAV_EALS,HWAV_EALR,HWAV_ALRB,HWAV_ALRP,HWAV_ALRA,HWAV_ALR_NPCTL,HWAV_RISKV,HWAV_RISKS,HWAV_RISKR,HRCN_EVNTS,HRCN_AFREQ,HRCN_EXP_AREA,HRCN_EXPB,HRCN_EXPP,HRCN_EXPPE,HRCN_EXPA,HRCN_EXPT,HRCN_HLRB,HRCN_HLRP,HRCN_HLRA,HRCN_HLRR,HRCN_EALB,HRCN_EALP,HRCN_EALPE,HRCN_EALA,HRCN_EALT,HRCN_EALS,HRCN_EALR,HRCN_ALRB,HRCN_ALRP,HRCN_ALRA,HRCN_ALR_NPCTL,HRCN_RISKV,HRCN_RISKS,HRCN_RISKR,ISTM_HLRR,ISTM_EALR,ISTM_RISKR,LNDS_EVNTS,LNDS_AFREQ,LNDS_EXP_AREA,LNDS_EXPB,LNDS_EXPP,LNDS_EXPPE,LNDS_EXPT,LNDS_HLRB,LNDS_HLRP,LNDS_HLRR,LNDS_EALB,LNDS_EALP,LNDS_EALPE,LNDS_EALT,LNDS_EALS,LNDS_EALR,LNDS_ALRB,LNDS_ALRP,LNDS_ALR_NPCTL,LNDS_RISKV,LNDS_RISKS,LNDS_RISKR,LTNG_EVNTS,LTNG_AFREQ,LTNG_EXP_AREA,LTNG_EXPB,LTNG_EXPP,LTNG_EXPPE,LTNG_EXPT,LTNG_HLRB,LTNG_HLRP,LTNG_HLRR,LTNG_EALB,LTNG_EALP,LTNG_EALPE,LTNG_EALT,LTNG_EALS,LTNG_EALR,LTNG_ALRB,LTNG_ALRP,LTNG_ALR_NPCTL,LTNG_RISKV,LTNG_RISKS,LTNG_RISKR,RFLD_EVNTS,RFLD_AFREQ,RFLD_EXP_AREA,RFLD_EXPB,RFLD_EXPP,RFLD_EXPPE,RFLD_EXPA,RFLD_EXPT,RFLD_HLRB,RFLD_HLRP,RFLD_HLRA,RFLD_HLRR,RFLD_EALB,RFLD_EALP,RFLD_EALPE,RFLD_EALA,RFLD_EALT,RFLD_EALS,RFLD_EALR,RFLD_ALRB,RFLD_ALRP,RFLD_ALRA,RFLD_ALR_NPCTL,RFLD_RISKV,RFLD_RISKS,RFLD_RISKR,SWND_EVNTS,SWND_AFREQ,SWND_EXP_AREA,SWND_EXPB,SWND_EXPP,SWND_EXPPE,SWND_EXPA,SWND_EXPT,SWND_HLRB,SWND_HLRP,SWND_HLRA,SWND_HLRR,SWND_EALB,SWND_EALP,SWND_EALPE,SWND_EALA,SWND_EALT,SWND_EALS,SWND_EALR,SWND_ALRB,SWND_ALRP,SWND_ALRA,SWND_ALR_NPCTL,SWND_RISKV,SWND_RISKS,SWND_RISKR,TRND_EVNTS,TRND_AFREQ,TRND_EXP_AREA,TRND_EXPB,TRND_EXPP,TRND_EXPPE,TRND_EXPA,TRND_EXPT,TRND_HLRB,TRND_HLRP,TRND_HLRA,TRND_HLRR,TRND_EALB,TRND_EALP,TRND_EALPE,TRND_EALA,TRND_EALT,TRND_EALS,TRND_EALR,TRND_ALRB,TRND_ALRP,TRND_ALRA,TRND_ALR_NPCTL,TRND_RISKV,TRND_RISKS,TRND_RISKR,TSUN_EVNTS,TSUN_AFREQ,TSUN_EXP_AREA,TSUN_EXPB,TSUN_EXPP,TSUN_EXPPE,TSUN_EXPT,TSUN_HLRB,TSUN_HLRP,TSUN_HLRR,TSUN_EALB,TSUN_EALP,TSUN_EALPE,TSUN_EALT,TSUN_EALS,TSUN_EALR,TSUN_ALRB,TSUN_ALRP,TSUN_ALR_NPCTL,TSUN_RISKV,TSUN_RISKS,TSUN_RISKR,VLCN_EVNTS,VLCN_AFREQ,VLCN_EXP_AREA,VLCN_EXPB,VLCN_EXPP,V

In [134]:
df_cleaned = final_merged_data.drop(columns='geometry', errors='ignore')  # Drop geometry column if you don't need it


In [135]:
# merging the sentences into a txt file


# taking the "sentence" column and joining all entries into a single text string, with each sentence on a newline
sentences = df_cleaned['sentence'].dropna()  # Remove any missing values
combined_text = "\n".join(sentences)

# saving the combined text into a .txt file
output_file_path = 'merged_sentences.txt'
with open(output_file_path, 'w') as file:
    file.write(combined_text)

# downloading the merged text file
from google.colab import files
files.download(output_file_path)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>